In [5]:
import pandas as pd
from random_address import real_random_address
import random

def str_address() -> str:
    addr = real_random_address()
    return addr["address1"] + " " + addr["state"] + " " + addr["postalCode"]

def any_match(l, a, v) -> bool:
    for el in l:
        if el[a] == v:
            return True
    
    return False

def where_match(l, a, v) -> int:
    for i, el in enumerate(l):
        if el[a] == v:
            return i
    
    return False

# no modifikation needed
studiengaenge = pd.read_excel('./Mockup_Wissenschaftliche_Arbeiten.xlsx', sheet_name="Studiengaenge")
fachbereiche = pd.read_excel('./Mockup_Wissenschaftliche_Arbeiten.xlsx', sheet_name="Fachbereiche")

studyprograms = studiengaenge.to_dict(orient="records")
semester = pd.read_excel('./Mockup_Wissenschaftliche_Arbeiten.xlsx', sheet_name="Semester")

# split arbeiten table into arbeiten, studenten, referenten
combined = pd.read_excel('./Mockup_Wissenschaftliche_Arbeiten.xlsx', sheet_name="Arbeiten")
combined.drop(combined.tail(1).index, inplace=True)
combined = combined.to_dict(orient="records")

studenten = []
referenten = []
arbeiten = []

ref_idc = 0

for i, combination in enumerate(combined):
    studenten.append({
        "id": i,
        "firstName": combination["Vorname Studierende/r"],
        "lastName": combination["Nachname Studierende/r"],
        "address": str_address(),
        "email": combination["Kontakt Studierende"],
        "phoneNumber": "+49 " + "".join([str(random.randint(1, 9)) for _ in range(10)]),
        "studentNumber": combination["Matrikelnumme"],
        "academicLevel": random.choice(["Bachelor", "Master"]),
        "scientificWorksIds": [i]
    })

    arbeiten.append({
        "id": i,
        "colloquium": None,
        "title": combination["Titel"],
        "startDate": combination["Startdatum"],
        "endDate": combination["Abgabedatum"],
        "studentId": i,
        "studyProgramId": where_match(studyprograms, "TitelNachAbschluss", combination["Studiengang"]),
    })

    if not any_match(referenten, "firstName", combination["Erstprüfer"]):
        referenten.append({
            "id": ref_idc,
            "firstName": combination["Erstprüfer"],
            "lastName": None,
            "email": combination["Erstprüfer"].replace(" ", "").lower() + "@thm.de",
            "phoneNumber": "+49 " + "".join([str(random.randint(1, 9)) for _ in range(10)]),
            "academicLevel": None,
            "role": None
        })

        ref_idc += 1

    if not any_match(referenten, "firstName", combination["Zweitprüfer"]):
        referenten.append({
            "id": ref_idc,
            "firstName": combination["Zweitprüfer"],
            "lastName": None,
            "email": combination["Zweitprüfer"].replace(" ", "").lower() + "@thm.de",
            "phoneNumber": "+49 " + "".join([str(random.randint(1, 9)) for _ in range(10)]),
            "academicLevel": None,
            "role": None
        })

        ref_idc += 1

# pd.DataFrame(studenten).to_json("./mock-prototyp/studenten.json", orient="records")
# pd.DataFrame(referenten).to_json("./mock-prototyp/referenten.json", orient="records")
# pd.DataFrame(arbeiten).to_json("./mock-prototyp/arbeiten.json", orient="records")

# studiengaenge.to_json("./mock-prototyp/studiengaenge.json", orient="records")
# semester.to_json("./mock-prototyp/semester.json", orient="records")
# fachbereiche.to_json("./mock-prototyp/fachbereiche.json", orient="records")

# studiengaenge = studiengaenge.rename(columns={"TitelNachAbschluss": "title"})
# sg_titel = studiengaenge[['title']].to_dict(orient="records")
# sg_titel = [{'title': e['title'], 'sws': 0.2, 'degreeType': e['title'].split(" ")[0].replace(".", "_", 1).replace(".", "", 1).upper()} for e in sg_titel]
# pd.DataFrame(sg_titel).to_json("./mock-insert/studiengaenge_1.json", orient="records")

titel = [arbeit["title"] for arbeit in arbeiten]
pd.DataFrame(titel).to_json("./mock-insert/titel.json", orient="values")